<a href="https://colab.research.google.com/github/1231232844/1231232844/blob/main/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.安装依赖

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton,
!pip install --no-deps cut_cross_entropy unsloth_zoo,
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer,
!pip install --no-deps unsloth

In [ ]:
from huggingface_hub import login,
from google.colab import userdata,
hf_token=userdata.get('hugging-key'),
login(hf_token)

In [ ]:
import wandb
wb_token = userdata.get("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B',
    job_type="training",
    anonymous="allow"
)

In [ ]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch
# 资源条件>=24GPU 可选择14/32B\n",

max_seq_length = 2048
dtype = None
load_in_4bit = True
hf_token=userdata.get('hugging-key')
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
  )

<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W&B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{sweeps-video} -->

<img src="https://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{sweeps-video} -->

<div><img /></div>

<img src="https://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />

<div><img /></div>